# Векторизованные строковые операции (Vectorized String Operations)

## Введение в строковые операции в Pandas

Векторизация (vectorization) операции упрощает синтакс операций на массивах данных: мы можем больше не беспокоиться о размере или форме массива, а сконцентрироваться на операции, которую мы хотим выполнить.

In [2]:
data = ['peter', 'Paul', 'MARY', 'gUIDO']
# Пример веторизованной операции на строковых данных в массиве
[s.capitalize() for s in data]

['Peter', 'Paul', 'Mary', 'Guido']

In [3]:
import pandas as pd
names = pd.Series(data)
names

0    peter
1     Paul
2     MARY
3    gUIDO
dtype: object

Так как Pandas позволяет работать даже с отсутствующими данными, то мы можем использовать эту возможность, чтобы капитализировать (capitalize) строковые данные:

In [6]:
names.str.capitalize()

0    Peter
1     Paul
2     Mary
3    Guido
dtype: object

## Таблицы строковых методов Pandas

Далее в этом разделе мы будем использовать такой набор данных:

In [13]:
monte = pd.Series(
    ['Graham Chapman', 'John Cleese', 'Terry Gilliam', 'Eric Idle', 'Terry Jones', 'Michael Palin']
)

### Методы идентичные строковым методам Python

Почти все встроенные строковые методы Python доступны в Pandas как векторизованные строковоые методы. Список методов в Pandas:

|           |                |                |                |
|-----------|----------------|----------------|----------------|
|`len()`    | `lower()`      | `translate()`  | `islower()`    | 
|`ljust()`  | `upper()`      | `startswith()` | `isupper()`    | 
|`rjust()`  | `find()`       | `endswith()`   | `isnumeric()`  | 
|`center()` | `rfind()`      | `isalnum()`    | `isdecimal()`  | 
|`zfill()`  | `index()`      | `isalpha()`    | `split()`      | 
|`strip()`  | `rindex()`     | `isdigit()`    | `rsplit()`     | 
|`rstrip()` | `capitalize()` | `isspace()`    | `partition()`  | 
|`lstrip()` | `swapcase()`   | `istitle()`    | `rpartition()` |

В отличие от Python эти методы могут иметь другие возвращаемые типы. Некоторые методы, как `lower()`, возвращают серию строк:

In [14]:
monte.str.lower()

0    graham chapman
1       john cleese
2     terry gilliam
3         eric idle
4       terry jones
5     michael palin
dtype: object

Некоторые другие методы возвращают числа:

In [16]:
monte.str.len()

0    14
1    11
2    13
3     9
4    11
5    13
dtype: int64

Некоторые возвращают логические значения:

In [18]:
monte.str.startswith("T")

0    False
1    False
2     True
3    False
4     True
5    False
dtype: bool

Некоторые методы возвращают списки или другие составные значения для каждого элемента:

In [20]:
monte.str.split()

0    [Graham, Chapman]
1       [John, Cleese]
2     [Terry, Gilliam]
3         [Eric, Idle]
4       [Terry, Jones]
5     [Michael, Palin]
dtype: object

### Методы с использованием регулярных выражений

В дополнение, существуют методы, которые принимают регулярное выражение для исследования содержимого каждого строкового элемента, и (методы) следуют соглашениям API принятым во встроенном в Python модуле `re`:

| Method    | Description |
|-----------|-------------|
| `match`   | Calls `re.match` on each element, returning a Boolean. |
| `extract` | Calls `re.match` on each element, returning matched groups as strings.|
| `findall` | Calls `re.findall` on each element |
| `replace` | Replaces occurrences of pattern with some other string|
| `contains`| Calls `re.search` on each element, returning a boolean |
| `count`   | Counts occurrences of pattern|
| `split`   | Equivalent to `str.split`, but accepts regexps |
| `rsplit`  | Equivalent to `str.rsplit`, but accepts regexps |

In [21]:
# Пример использования

monte.str.extract('([A-Za-z]+)', expand=False)

0     Graham
1       John
2      Terry
3       Eric
4      Terry
5    Michael
dtype: object

### Другие методы

| Method | Description |
|--------|-------------|
| `get` | Indexes each element |
| `slice` | Slices each element|
| `slice_replace` | Replaces slice in each element with the passed value|
| `cat`      | Concatenates strings|
| `repeat` | Repeats values |
| `normalize` | Returns Unicode form of strings |
| `pad` | Adds whitespace to left, right, or both sides of strings|
| `wrap` | Splits long strings into lines with length less than a given width|
| `join` | Joins strings in each element of the `Series` with the passed separator|
| `get_dummies` | Extracts dummy variables as a `DataFrame` |

#### Векторизованный доступ к элементу и срезы

In [25]:
# Срез каждого элемента по первым трём символам
monte.str[0:3]

0    Gra
1    Joh
2    Ter
3    Eri
4    Ter
5    Mic
dtype: object

## Пример: база рецептов

Мы попробуем отпарсить рецепты и составить список ингридиентов.

In [28]:
try:
    recipes = pd.read_json('notebooks/data/recipeitems.json')
except ValueError as e:
    print("ValueError:", e)

ValueError: Trailing data


Ошибка `ValueError: Trailing data` говорит о том, что файл является JSONlines.

In [30]:
with open('notebooks/data/recipeitems.json') as f:
    line = f.readline()
pd.read_json(line).shape

/tmp/ipykernel_18873/771231573.py:3: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_json(line).shape


(2, 12)

In [43]:
import io
# read the entire file into a Python array
with open('notebooks/data/recipeitems.json', 'r') as f:
    # Extract each line
    data = (line.strip() for line in f)
    # Reformat so each line is the element of a list
    data_json = "[{0}]".format(','.join(data))
# read the result as a JSON
recipes = pd.read_json(io.StringIO(data_json))

In [44]:
recipes.shape

(173278, 17)

In [45]:
recipes.iloc[0]

_id                                {'$oid': '5160756b96cc62079cc2db15'}
name                                    Drop Biscuits and Sausage Gravy
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
url                   http://thepioneerwoman.com/cooking/2013/03/dro...
image                 http://static.thepioneerwoman.com/cooking/file...
ts                                             {'$date': 1365276011104}
cookTime                                                          PT30M
source                                                  thepioneerwoman
recipeYield                                                          12
datePublished                                                2013-03-11
prepTime                                                          PT10M
description           Late Saturday afternoon, after Marlboro Man ha...
totalTime                                                           NaN
creator                                                         

In [46]:
recipes.ingredients.str.len().describe()

count    173278.000000
mean        244.617926
std         146.705285
min           0.000000
25%         147.000000
50%         221.000000
75%         314.000000
max        9067.000000
Name: ingredients, dtype: float64

In [48]:
import numpy as np
# Рецепт с наиболее длинным списком ингридиентов
recipes.name[np.argmax(recipes.ingredients.str.len())]

'Carrot Pineapple Spice &amp; Brownie Layer Cake with Whipped Cream &amp; Cream Cheese Frosting and Marzipan Carrots'

In [49]:
# Сколько рецептов с завтраками
recipes.description.str.contains('[Bb]reakfast').sum()

3524

In [50]:
# Сколько рецептов с корицей
recipes.ingredients.str.contains('[Cc]innamon').sum()

10526

In [51]:
# Сколько рецептов с неправильным написанием слова "корица"
recipes.ingredients.str.contains('[Cc]inamon').sum()

11

### Простая система рекомендации рецептов

https://jakevdp.github.io/PythonDataScienceHandbook/03.10-working-with-strings.html#A-simple-recipe-recommender